In [21]:
import sys
sys.path.append("../../src/")
import numpy as np
import math
import uncertainpy.gradual as grad

In [22]:
def compute_removal(topic_a, cur_arg):

    model = grad.semantics.QuadraticEnergyModel()
    model.approximator = grad.algorithms.RK4(model)
    model.BAG = grad.BAG("../../bags/td_qbaf.bag")
    model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)

    arg_name_all = []

    for arg in model.BAG.arguments.values():
        arg_name_all.append(arg.name)
        # print((arg.name,arg.strength))
    arg_name_no_topic_arg = arg_name_all.copy()
    arg_name_no_topic_arg.remove(topic_a)

    print(arg_name_all)
    print(arg_name_no_topic_arg)

    # remove current argument
    arg_name_no_topic_no_cur = arg_name_no_topic_arg.copy()
    arg_name_no_topic_no_cur.remove(cur_arg)
    print(arg_name_no_topic_no_cur)
    print("----------------------------")


    # store the initial weights
    arg_initial_weight = []
    # arg_initial_weight_dict = {}
    for arg in model.BAG.arguments.values():
        arg_initial_weight.append((arg.name,arg.initial_weight))

    print("All the base score and strength in QBAF:")
    for arg in model.BAG.arguments.values():
        print((arg.name,arg.initial_weight,arg.strength))


    difference = []

    # print 1st: before remove cur arg
    print("Initial and final strength (1st):")
    for arg in model.BAG.arguments.values():
        print((arg.name,arg.initial_weight,arg.strength))
    topic_arg = model.BAG.arguments[topic_a]
    strength_before_drop_cur = topic_arg.strength
    print(f"strength_before_drop_cur:{strength_before_drop_cur}")

    # remove cur arg
    model.BAG.remove_argument(cur_arg)
    model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)

    # print 2nd: after remove cur arg
    print("Initial and final strength (2nd):")
    for arg in model.BAG.arguments.values():
        print((arg.name,arg.initial_weight,arg.strength))
    topic_arg = model.BAG.arguments[topic_a]
    strength_after_drop_cur = topic_arg.strength
    print(f"strength_after_drop_cur:{strength_after_drop_cur}")

    # compute the diff of topic arg before and after removing
    difference.append(strength_before_drop_cur-strength_after_drop_cur)
    print(f"diff={strength_before_drop_cur-strength_after_drop_cur}")



    print("----------------------------")


    print(difference)
    attribution = difference[0]
    print(attribution)

    # initializing: reset to original initial weight
    model = grad.semantics.QuadraticEnergyModel()
    model.approximator = grad.algorithms.RK4(model)
    model.BAG = grad.BAG("../../bags/td_qbaf.bag")
    model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)

    return attribution

In [23]:
model = grad.semantics.QuadraticEnergyModel()
model.approximator = grad.algorithms.RK4(model)
model.BAG = grad.BAG("../../bags/td_qbaf.bag")
model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)

QuadraticEnergyModel, RK4
Time: 12.199999999999973
Argument(name=s0, weight=0.5, strength=0.664592759656642)
Argument(name=s1, weight=0.5, strength=0.664592759656642)
Argument(name=s2, weight=0.5, strength=0.664592759656642)
Argument(name=s3, weight=0.5, strength=0.8135424356888933)
Argument(name=s4, weight=0.5, strength=0.7041994642766211)
Argument(name=s5, weight=0.5, strength=0.7041994642766211)
Argument(name=s6, weight=0.5, strength=0.5000000446420562)
Argument(name=s7, weight=0.5, strength=0.8254237055377077)
Argument(name=s8, weight=0.5, strength=0.7235959804104234)
Argument(name=s9, weight=0.5, strength=0.7235959804104234)
Argument(name=s10, weight=0.5, strength=0.7235959804104234)
Argument(name=c0, weight=0.0, strength=0.7000991119110755)
Argument(name=c1, weight=0.0, strength=0.46753135307445565)
Argument(name=c2, weight=0.0, strength=0.8313656875032375)
Argument(name=c3, weight=0.0, strength=4.617127656482882e-07)
Argument(name=c4, weight=0.0, strength=3.570538632983414e-07)


0.0009878603392623568

In [24]:
# compute attribution for one arg
topic_a = "c5"
all_args = model.BAG.get_arguments()
all_args_names = []
for i in all_args:
    all_args_names.append(i.get_name())
print(all_args_names)

all_args_names_without_topic = all_args_names.copy()
all_args_names_without_topic.remove(topic_a)
print(all_args_names_without_topic)

attribution = {}
for cur_arg in all_args_names_without_topic:
    attribution[cur_arg]=compute_removal(topic_a, cur_arg)
print(f"attribution: \n{attribution}")

['s0', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 'c0', 'c1', 'c2', 'c3', 'c4', 'c5']
['s0', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 'c0', 'c1', 'c2', 'c3', 'c4']
QuadraticEnergyModel, RK4
Time: 12.199999999999973
Argument(name=s0, weight=0.5, strength=0.664592759656642)
Argument(name=s1, weight=0.5, strength=0.664592759656642)
Argument(name=s2, weight=0.5, strength=0.664592759656642)
Argument(name=s3, weight=0.5, strength=0.8135424356888933)
Argument(name=s4, weight=0.5, strength=0.7041994642766211)
Argument(name=s5, weight=0.5, strength=0.7041994642766211)
Argument(name=s6, weight=0.5, strength=0.5000000446420562)
Argument(name=s7, weight=0.5, strength=0.8254237055377077)
Argument(name=s8, weight=0.5, strength=0.7235959804104234)
Argument(name=s9, weight=0.5, strength=0.7235959804104234)
Argument(name=s10, weight=0.5, strength=0.7235959804104234)
Argument(name=c0, weight=0.0, strength=0.7000991119110755)
Argument(name=c1, weight=0.0, strength=0.